<a href="https://colab.research.google.com/github/bronte999/VegMapper/blob/master/GEDI_ShotProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GEDI Shot Processing


Script to process and filter shot data from  "Footprint level canopy height and profile metrics" dataset located at: https://gedi.umd.edu/data/products/ 


Requirements: 

Access to bucket (use EC2 or other trusted entity that can read/write) (link to tutorial)

In [1]:
%pip install h5py numpy pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import h5py
import numpy as np
import pandas as pd
import glob
import itertools
import statistics
import time
#import gsutil

pd.options.mode.chained_assignment = None
import warnings
warnings.filterwarnings("ignore")

##User input
These are global variables defined by the user of this code. Please set these before running your own experiment. 

`h5FilesToProcess`: Specify location of text file containing a list of the names of h5 files to process.

`sourceDirectory`: Specify the directory where the h5 files are located. This must be located in a bucket (either cloud or S3) that user can read/write to in current environment. 

`destinationFilePrefix`: Prefix to append to individual csv files after extracting data from h5 file. For example, if processing `GEDI02_A_2020198095831_O09023_T00771_02_001_01.h5` the resulting csv file will be `PREFIX_GEDI02_A_2020198095831_O09023_T00771_02_001_01.csv`

`destinationDirectory`: Directory where individual csv files and final aggregated csv file will be saved. This doesn't have to be a bucket. 

`outputFileName`: Name of the final aggregated csv file. This is contains all of the data from each of the individual csv files generated. 

In [ ]:
h5FilesToProcess = "h5S3files.txt"
sourceDirectory = "s3://servir-public/gedi/peru/"
destinationFilePrefix = "filtered_"
destinationDirectory = "s3://servir-public/gedi/peru/"
outputFileName = "FILTERED_CSV.csv"

## The Process:
For each h5 file listed in `h5FilesToProcess`...

1. Download the file
2. Extract and filter data for each beam in the shot file
3. Save filtered data for all beams in the shot in csv 
4. Delete downloaded file and repeat 1-3
5. Concatenate/aggregate all shot data in one csv

##Main

Sets up experiment constants and 

In [ ]:
def main():
    csv_files = [] ## list that keeps track of all csv files generated

    ##generate column names for rh vals (ranges 1-100) 
    rh_cols = []
    for i in range(101):
      rh_cols.append('rh'+str(i))

    readH5Files(csv_files, rh_cols)
    print("FINISHED")

##readH5Files
Downloads one h5 file at a time and reads it. If the file is not an h5 file or is corrupted (can't be read) the downloaded file is deleted and the next file is downloaded to be read. Once the file is downloaded and verified to be a readable h5 file, processBeams is called. 

In [ ]:

def readH5Files(csv_files, rh_cols):
  f = open(h5FilesToProcess, "r")

  for h5file in f.readlines(): 
      h5file = h5file.strip()

      #download the h5file from aws
      cpS3url = "aws s3 cp "+ sourceDirectory+ + h5file + " ./"
      
      print("DOWNLOAD FILE: " + cpS3url)
      os.system(cpS3url)

      ###READ DATA
      try:
          gediL2A = h5py.File(h5file, 'r')
      except Exception as e:
          print(e)
          os.remove(h5file)
      else:
        processBeams(gediL2A, h5file, csv_files, rh_cols)


##processBeams
Each shot file contains information for multiple beams. Data will be extracted from each beam (`extractBeamData()`), one at a time, and then combined to be filtered by `filterBeamData`. 

In [ ]:
def processBeams(gediL2A, h5file, csv_files, rh_cols):
    ## approx 4 beams in every file
    beamNames = [g for g in gediL2A.keys() if g.startswith('BEAM')]
    beam_dfs = [] #create list to hold resulting dataframes for each beam

    gediL2A_objs = []
    gediL2A.visit(gediL2A_objs.append)                                           # Retrieve list of datasets
    gediSDS = [o for o in gediL2A_objs if isinstance(gediL2A[o], h5py.Dataset)]  # Search for relevant SDS inside data file 

    for highBeam in beamNames: #extract data from each beam in the shot
      beam_df = extractBeamData(highBeam, gediL2A, rh_cols)
      if beam_df: #if beam data extraction is successful, append resulting dataframe to list 
        beam_dfs.append(beam_df)
    
    if len(beam_dfs) >= 2:
      all_beam_dfs = pd.concat(beam_dfs) #combine all beam dataframes
      filterBeamData(all_beam_dfs, h5file, csv_files)
    elif len(beam_dfs) == 1:
      filterBeamData(beam_dfs, h5file, csv_files)
    else:
      return

##extractBeamData
Return selected beam features in pandas dataframe.  

In [ ]:
def extractBeamData(highBeam, gediL2A, rh_cols):
        gedi_beam_data = {}
        try:
            gedi_beam_data["lats"] = gediL2A[f'{highBeam}/lat_lowestmode'][()]
            gedi_beam_data["long"] = gediL2A[f'{highBeam}/lon_lowestmode'][()]
            gedi_beam_data["shotNumber"] = gediL2A[f'{highBeam}/shot_number'][()]
            gedi_beam_data["quality"] = gediL2A[f'{highBeam}/quality_flag'][()]
            gedi_beam_data["solar_elev"] = gediL2A[f'{highBeam}/solar_elevation'][()]
            gedi_beam_data["sensitivity"] = gediL2A[f'{highBeam}/sensitivity'][()]
            #gedi_beam_data["rh"] = gediL2A[f'{highBeam}/rh'][()]
            gedi_beam_data["elevLow"] = gediL2A[f'{highBeam}/elev_lowestmode'][()]

            gedi_beam_data["delta_time"] = gediL2A[f'{highBeam}/delta_time'][()]
            gedi_beam_data["energy_total"] = gediL2A[f'{highBeam}/energy_total'][()]
                
            gedi_beam_data["rx_gbias"] = gediL2A[f'{highBeam}/rx_1gaussfit/rx_gbias'][()]
            gedi_beam_data["rx_gamplitude_error"] = gediL2A[f'{highBeam}/rx_1gaussfit/rx_gamplitude_error'][()]
                
            gedi_beam_data["rx1_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a1/energy_sm'][()]
            gedi_beam_data["rx2_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a2/energy_sm'][()]
            gedi_beam_data["rx3_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a3/energy_sm'][()]
            gedi_beam_data["rx4_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a4/energy_sm'][()]
            gedi_beam_data["rx5_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a5/energy_sm'][()]
            gedi_beam_data["rx6_energy_sm"] = gediL2A[f'{highBeam}/rx_processing_a6/energy_sm'][()]
             
            gedi_beam_data["rx1_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a1/lastmodeenergy'][()]
            gedi_beam_data["rx2_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a2/lastmodeenergy'][()]
            gedi_beam_data["rx3_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a3/lastmodeenergy'][()]
            gedi_beam_data["rx4_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a4/lastmodeenergy'][()]
            gedi_beam_data["rx5_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a5/lastmodeenergy'][()]
            gedi_beam_data["rx6_lastmodeenergy"] = gediL2A[f'{highBeam}/rx_processing_a6/lastmodeenergy'][()]
                
            gedi_beam_data["elev1_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a1'][()]
            gedi_beam_data["elev2_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a2'][()]
            gedi_beam_data["elev3_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a3'][()]
            gedi_beam_data["elev4_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a4'][()]
            gedi_beam_data["elev5_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a5'][()]
            gedi_beam_data["elev6_low_energy"] = gediL2A[f'{highBeam}/geolocation/energy_lowestmode_a6'][()]
                
            gedi_beam_data["elev1_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a1'][()]
            gedi_beam_data["elev2_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a2'][()]
            gedi_beam_data["elev3_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a3'][()]
            gedi_beam_data["elev4_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a4'][()]
            gedi_beam_data["elev5_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a5'][()]
            gedi_beam_data["elev6_num_modes"] = gediL2A[f'{highBeam}/geolocation/num_detectedmodes_a6'][()]
             
            gedi_beam_data["elev1"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a1'][()]
            gedi_beam_data["elev2"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a2'][()]
            gedi_beam_data["elev3"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a3'][()]
            gedi_beam_data["elev4"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a4'][()]
            gedi_beam_data["elev5"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a5'][()]
            gedi_beam_data["elev6"] = gediL2A[f'{highBeam}/geolocation/elev_lowestmode_a6'][()]
            rh = gediL2A[f'{highBeam}/rh'][()]
            gedi_beam_data["beam"] = np.full(len(gedi_beam_data['shotNumber']), highBeam)

            raw_df = pd.DataFrame(gedi_beam_data)
            raw_df[rh_cols] = pd.DataFrame(rh, index= raw_df.index) #add rh values
            raw_df = raw_df.dropna() 
        except Exception as e:
            print("EXCEPTION: ", e)
            return None
        else:
          return raw_df

##rangeCalculator
This function is used to calculate the range of elevations between elevation modes. Only considers values that are within 2 standarad deviations of the mean. If none of the values are are within 2 standard deviations of the mean then a dummy value is returned to flag this row to be discarded. This method ensures that the range is not affected by outlier elevation observations.  

In [ ]:
## returns the range between valid elevations (elevations that are within 2SD of the mean)
def rangeCalculator(elevList, sd, mean):
    elevList = [item for item in elevList if item <= (mean + (2*sd)) and item >= (mean - (2*sd)) ]
    if elevList:
        return max(elevList) - min(elevList)
    else: 
        ##all elevations are abnormal, return -99 to indicate data from this row should not be included in analysis
        return 99

##filterBeamData
After extracting the data from each beam, the beam data needs to be filtered. The criteria we are filtering on is as follows:
1. Sensitivity greater than 90%
2. Elevation range (from rangeCalculator) is less than 2
3. rh95 is greater than 0 (not ground point)

Once this data is filtered, `saveFilteredData` is called. Data is only saved if filtering is successful, if an error occurs the process does not continue. 

In [ ]:
def filterBeamData(all_beam_dfs, h5file, csv_files):
  try:
    filtered =  all_beam_dfs[all_beam_dfs.sensitivity >= 0.9] #sensitivity > 90%
          
    #filter based on elevation
    elevs = pd.concat([filtered.elev1, filtered.elev2, filtered.elev3, filtered.elev4, filtered.elev5, filtered.elev6])
    filtered['elev_sd'] = elevs.std()
    filtered['elev_mean'] = elevs.mean() #calculate mean of elevs
    filtered['elev_range'] = filtered.apply(lambda x: rangeCalculator([x['elev1'], x['elev2'], x['elev3'], x['elev4'], x['elev5'], x['elev6']], x['elev_sd'], x['elev_mean']), axis = 1)
          
    GEDI_DF = filtered[abs(filtered.elev_range) < 2] #only select rows that have valid ranges

    GEDI_DF = GEDI_DF[GEDI_DF.rh95 > 0] ## excludes ground points
  except Exception as e:
    print("EXCEPTION: ", e)
    return
  else:
    saveFilteredData(GEDI_DF, h5file, csv_files)

##saveFilteredData
Saves shot data to an individual csv file in the current directory, then moves the file to the destination directory. The name of the csv file is the name of the h5 file and the user specified prefix. The processed h5 file is then deleted from the local directory to free up space. 

In [ ]:
def saveFilteredData(GEDI_DF, h5file, csv_files):
  #save final filtered dataframe to a csv file in the destination directory with the correct prefix
  csv_file = destinationFilePrefix + h5file[:-3]+".csv"
  csv_files.append(csv_file)
  print("CREATING CSV: " + csv_file)
  GEDI_DF.to_csv(csv_file)
  cp_csv = "aws s3 cp " + csv_file + " " + destinationDirectory + csv_file
  print("COPY CSV TO S3: " + cp_csv)
  os.system(cp_csv)
  os.remove(csv_file)
  try:
    os.remove(h5file)
  except Exception as e:
    print("EXCEPTION: ,", e)

The entire process of downloading the raw shot data, extracting data from individual beams, filtering the sho tdata, and saving the result repeats for the next h5 file listed in the `h5FileFilesToProcess` 

##Concatenating the Results
This step is optional. The individual csv files generated for each h5 file can be concatenated into a single csv file. This is not memory optimized and may not be feasible for larger areas of interest. 

In [ ]:
files_to_concat = []
for f in csv_files:
    local_filename = f.replace(destinationDirectory,'') #remove path info, string should now be of format: prefix_name.csv

    cpS3url = "aws s3 cp " + f +  " " + local_filename #download csv file from destination directory, save to local
    print(cpS3url)
    os.system(cpS3url)

    csv_df =  pd.read_csv(f) #create df from file
    files_to_concat.append(csv_df) #append dataframe to list to concat later
    os.remove(local_filename) #remove file when done, save space 

df = pd.concat(files_to_concat) #concat all the files
df.to_csv(outputFileName, index = False) #create final dataframe